In [1]:
# Import the necessary modules
from ow_ustx import *
import json
import time
from pyfus.io.ustx import PulseProfile, DelayProfile, TxModule, TxArray, Tx7332Registers, print_regs
from pyfus.xdc import Transducer, Element
import numpy as np
import sys
from ow_ustx import *
import json
import time

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:matplotlib:matplotlib data path: c:\Users\gvigelet\.conda\envs\pyfus\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\gvigelet\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\gvigelet\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\gvigelet\.matplotlib\fontlist-v330.json


In [2]:
CTRL_BOARD = True
serial_obj = None

if CTRL_BOARD:
    vid = 1155  # Example VID for demonstration
    pid = 22446  # Example PID for demonstration
    
    com_port = list_vcp_with_vid_pid(vid, pid)
    if com_port is None:
        print("No device found")
    else:
        print("Device found at port: ", com_port)
        # Select communication port
        serial_obj = UART(com_port, timeout=5)
else:
    serial_obj = UART('COM16', timeout=5)

INFO:UART:Connecting to COM port at COM16 speed 921600


In [4]:
# Initialize the USTx controller object
ustx_ctrl = CTRL_IF(serial_obj)

print("USTx controller initialized")
r = ustx_ctrl.ping()  # Ping the device
try:
    parsedResp = UartPacket(buffer=r)
    print("Received From Controller Packet ID: ", parsedResp.id)
except ValueError as e:
    print("{0}".format(e))
    sys.exit(0)

USTx controller initialized
Received From Controller Packet ID:  1


In [5]:
# enumerate devices
print("Enumerate I2C Devices")
ustx_ctrl.enum_i2c_devices()
print("Enumerate TX7332 Chips on AFE devices")
for afe_device in ustx_ctrl.afe_devices:
    print("Enumerate TX7332 Chips on AFE device")
    rUartPacket = afe_device.enum_tx7332_devices()

Enumerate I2C Devices
Enumerate TX7332 Chips on AFE devices
Enumerate TX7332 Chips on AFE device
Status Packet:
  ID: 3
  Command: 0x31
  Status: 0x0
  Reserved: 0x2
  Data Length: 8
  CRC: 0x7dc7


In [6]:
# Set the focus and pulse profile
focus = np.array([0, 0, 50]) #set focus #left, front, down 
pulse_profile = PulseProfile(profile=1, frequency=400e3, cycles=2000)
first_afe = ustx_ctrl.afe_devices[0]
afe_dict = {first_afe.i2c_addr: first_afe}
# afe_dict = {afe.i2c_addr:afe for afe in ustx_ctrl.afe_devices}
# Load Mapping file
arr = Transducer.from_file(R"M3.json")
arr.elements = np.array(arr.elements)[np.argsort([el.pin for el in arr.elements])].tolist()
distances = np.sqrt(np.sum((focus - arr.get_positions(units="mm"))**2, 1))
tof = distances*1e-3 / 1500
delays = tof.max() - tof
i2c_addresses = list(afe_dict.keys())
i2c_addresses = i2c_addresses[:int(arr.numelements()/64)]
txa = TxArray(i2c_addresses=i2c_addresses)
array_delay_profile = DelayProfile(1, delays.tolist())
txa.add_delay_profile(array_delay_profile)
txa.add_pulse_profile(pulse_profile)
regs = txa.get_registers(profiles="configured", pack=True) 
for addr, rm in regs.items():
    print(f'I2C: 0x{addr:02x}')
    for i, r in enumerate(rm):
        print(f'MODULE {i}')
        print_regs(r)
    print('')  #calculate register state for 7332s, settings for board (bits, purpose), #change focus!!

INFO:root:Initializing transducer array


I2C: 0x34
MODULE 0
0x0:x00000000
0x1:x00000000
0x6:x00000000
0xB:x00000000
0xC:x00000000
0xF:x00000000
0x14:x00000000
0x15:x00000000
0x16:x00000000
0x18:x02000003
0x19:x04E20F40
0x1A:x00000000
0x1B:x00000000
0x1E:x00000000
0x1F:x00000000
0x20[+0]:x00610000
0x20[+1]:x009E003A
0x20[+2]:x0075003A
0x20[+3]:x00B20075
0x20[+4]:x007500B2
0x20[+5]:x003A0075
0x20[+6]:x003A009E
0x20[+7]:x00000061
0x20[+8]:x00C70061
0x20[+9]:x00DC0075
0x20[+10]:x00DC009E
0x20[+11]:x00F100B2
0x20[+12]:x007500F1
0x20[+13]:x009E00DC
0x20[+14]:x00B200DC
0x20[+15]:x006100C7
0x120[+0]:xF1D092F2
0x120[+1]:x0007D091
MODULE 1
0x0:x00000000
0x1:x00000000
0x6:x00000000
0xB:x00000000
0xC:x00000000
0xF:x00000000
0x14:x00000000
0x15:x00000000
0x16:x00000000
0x18:x02000003
0x19:x04E20F40
0x1A:x00000000
0x1B:x00000000
0x1E:x00000000
0x1F:x00000000
0x20[+0]:x007500F1
0x20[+1]:x009E00DC
0x20[+2]:x00B200DC
0x20[+3]:x006100C7
0x20[+4]:x00C70061
0x20[+5]:x00DC00B2
0x20[+6]:x00DC009E
0x20[+7]:x00F10075
0x20[+8]:x007500B2
0x20[+9]:x003

In [7]:
# Write Registers to Device #series of loops for programming tx chips 
for i2c_addr, module_regs in regs.items():
    afe = afe_dict[i2c_addr] 
    for i in range(len(module_regs)):
        device = afe.tx_devices[i]
        r = module_regs[i]
        device.write_register(0,1) #resetting the device
        for address, value in r.items():
            if isinstance(value, list):
                print(f"0x{i2c_addr:x}[{i}] Writing {len(value)}-value block starting at register 0x{address:X}")
                device.write_block(address, value)
            else:
                print(f"0x{i2c_addr:x}[{i}] Writing value 0x{value:X} to register 0x{address:X}")
                device.write_register(address, value)
            time.sleep(0.1)

0x34[0] Writing value 0x0 to register 0x0
0x34[0] Writing value 0x0 to register 0x1
0x34[0] Writing value 0x0 to register 0x6
0x34[0] Writing value 0x0 to register 0xB
0x34[0] Writing value 0x0 to register 0xC
0x34[0] Writing value 0x0 to register 0xF
0x34[0] Writing value 0x0 to register 0x14
0x34[0] Writing value 0x0 to register 0x15
0x34[0] Writing value 0x0 to register 0x16
0x34[0] Writing value 0x2000003 to register 0x18
0x34[0] Writing value 0x4E20F40 to register 0x19
0x34[0] Writing value 0x0 to register 0x1F
0x34[0] Writing value 0x0 to register 0x1E
0x34[0] Writing value 0x0 to register 0x1A
0x34[0] Writing value 0x0 to register 0x1B
0x34[0] Writing 16-value block starting at register 0x20
0x34[0] Writing 2-value block starting at register 0x120
0x34[1] Writing value 0x0 to register 0x0
0x34[1] Writing value 0x0 to register 0x1
0x34[1] Writing value 0x0 to register 0x6
0x34[1] Writing value 0x0 to register 0xB
0x34[1] Writing value 0x0 to register 0xC
0x34[1] Writing value 0x0

In [8]:
# get trigger configuration
trigger_config = ustx_ctrl.get_trigger()
print(json.dumps(trigger_config, indent=4))


{
    "TriggerFrequencyHz": 0,
    "TriggerMode": "UNKNOWN",
    "TriggerPulseCount": 0,
    "TriggerPulseWidthUsec": 0,
    "TriggerStatus": "NOT CONFIGURED"
}


In [9]:
# set trigger configuration
trigger_config = {
    "TriggerFrequencyHz": 50,
    "TriggerMode": 1,
    "TriggerPulseCount": 0,
    "TriggerPulseWidthUsec": 250
}
r = ustx_ctrl.set_trigger(data=trigger_config)
format_and_print_hex(r)

Received Data (Hex): AA 00 01 E3 13 00 00 00 00 26 0F DD


In [12]:
# start trigger pulse
print("Turn Trigger On")
ustx_ctrl.start_trigger()

Turn Trigger On


In [13]:
# stop trigger pulse
print("Turn Trigger Off")
ustx_ctrl.stop_trigger()

Turn Trigger Off


In [ ]:
# close teh communication port
ustx_ctrl.uart.close()